## This notebook test the semi-smooth Newton optimizer implemented with line-search and trust-region. 

In [1]:
%cd ..
%load_ext autoreload
%autoreload 2
# %load_ext nb_black

from pathlib import Path
import sys

sys.path.insert(0, Path().absolute().parent.as_posix())

/Users/ruizhechao/Documents/NNforHJB


/Users/ruizhechao/Documents/NNforHJB/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### 1d case: Gauss Window

In [8]:
import numpy as np

path = 'rawdata/raw_data/data/gaussian_window_1000.npy'
data = np.load(path)
data_dict = {
    "x": np.asarray(data["x"], dtype=np.float64),    # shape (N, 2)
    "v": np.asarray(data["v"], dtype=np.float64),    # shape (N,)
    "dv": np.asarray(data["dv"], dtype=np.float64),  # shape (N, 2)
}

1000

In [9]:
gamma = 5.0
alpha = 1e-5

In [10]:
from scr.model import model
import torch

test_ls = model(
    alpha, 
    gamma,
    optimizer='SSN',
    activation=torch.relu, 
    train_outerweights=True
    )

data_train, data_valid = test_ls._prepare_data(data_dict)

test_ls.train(data_train, data_valid)

2026-01-24 12:50:10.937 | INFO     | scr.model:__init__:72 - Model initialized
2026-01-24 12:50:10.938 | INFO     | scr.model:_prepare_data:107 - Training set: 900 samples, Validation set: 100 samples
2026-01-24 12:50:11.558 | INFO     | scr.model:_setup_optimizer:186 - Using SSN optimizer with alpha=1e-05, gamma=5.0, th=0.5, lr =1.0
2026-01-24 12:50:11.558 | INFO     | scr.model:train:265 - Starting network training session
2026-01-24 12:50:11.588 | INFO     | scr.model:train:330 - Epoch 0: Train Loss = 3.609354, Val Loss = 4.001666
2026-01-24 12:50:22.708 | INFO     | scr.model:train:330 - Epoch 1000: Train Loss = 1.949524, Val Loss = 2.358415
2026-01-24 12:50:33.831 | INFO     | scr.model:train:330 - Epoch 2000: Train Loss = 1.758677, Val Loss = 2.138960
2026-01-24 12:50:45.180 | INFO     | scr.model:train:330 - Epoch 3000: Train Loss = 1.431604, Val Loss = 1.850730
2026-01-24 12:50:56.649 | INFO     | scr.model:train:330 - Epoch 4000: Train Loss = 1.291596, Val Loss = 1.626716
2026

### 2d case:  Van der Pol equation

In [9]:
# load the data
import numpy as np

path = 'rawdata/raw_data/VDP_beta_0.1_grid_30x30.npy'
data = np.load(path)

# Shuffle once to avoid an ordered train/val split (deterministic)
rng = np.random.default_rng(0)
perm = rng.permutation(len(data))
data = data[perm]

data_dict = {
    "x": np.asarray(data["x"], dtype=np.float64),    # shape (N, 2)
    "v": np.asarray(data["v"], dtype=np.float64),    # shape (N,)
    "dv": np.asarray(data["dv"], dtype=np.float64),  # shape (N, 2)
}

#### SSN(trust-region) method for outer weights ##


In [13]:
from scr.model import model
import torch

test_ls = model(
    alpha, 
    gamma,
    optimizer='SSN',
    activation=torch.relu, 
    train_outerweights=True
    )

data_train, data_valid = test_ls._prepare_data(data_dict)

test_ls.train(data_train, data_valid)

2026-01-13 18:44:39.074 | INFO     | scr.model:__init__:71 - Model initialized
2026-01-13 18:44:39.075 | INFO     | scr.model:_prepare_data:101 - Training set: 810 samples, Validation set: 90 samples
2026-01-13 18:44:39.077 | INFO     | scr.model:_setup_optimizer:173 - Using SSN optimizer with alpha=1e-05, gamma=5.0, th=0.5, lr =1.0
2026-01-13 18:44:39.077 | INFO     | scr.model:train:252 - Starting network training session
2026-01-13 18:44:39.094 | INFO     | scr.model:train:316 - Epoch 0: Train Loss = 13.430782, Val Loss = 12.742445
2026-01-13 18:44:48.449 | INFO     | scr.model:train:316 - Epoch 1000: Train Loss = 2.186777, Val Loss = 2.422045
2026-01-13 18:44:58.171 | INFO     | scr.model:train:316 - Epoch 2000: Train Loss = 2.186777, Val Loss = 2.422040
2026-01-13 18:45:07.855 | INFO     | scr.model:train:316 - Epoch 3000: Train Loss = 2.186777, Val Loss = 2.422037
2026-01-13 18:45:17.591 | INFO     | scr.model:train:316 - Epoch 4000: Train Loss = 2.186777, Val Loss = 2.422031
202

In [15]:
test_tr = model(
    alpha, 
    gamma,
    optimizer='SSN_TR',
    activation=torch.relu, 
    train_outerweights=True
    )

data_train, data_valid = test_ls._prepare_data(data_dict)

test_tr.train(data_train, data_valid)

2026-01-13 18:46:14.285 | INFO     | scr.model:__init__:71 - Model initialized
2026-01-13 18:46:14.285 | INFO     | scr.model:_prepare_data:101 - Training set: 810 samples, Validation set: 90 samples
2026-01-13 18:46:14.287 | INFO     | scr.model:_setup_optimizer:173 - Using SSN_TR optimizer with alpha=1e-05, gamma=5.0, th=0.5, lr =1.0
2026-01-13 18:46:14.288 | INFO     | scr.model:train:252 - Starting network training session
2026-01-13 18:46:14.296 | INFO     | scr.model:train:316 - Epoch 0: Train Loss = 32.518156, Val Loss = 31.688785
2026-01-13 18:46:14.307 | DEBUG    | scr.ssn_tr:step:83 - TR reject: Δloss=5.329e-01 3.20e+00->6.40e-01
2026-01-13 18:46:14.308 | WARNING  | scr.model:train:299 - SSN step rejected (consecutive=1/3)
2026-01-13 18:46:14.314 | DEBUG    | scr.ssn_tr:step:83 - TR reject: Δloss=7.765e-01 2.56e+00->5.12e-01
2026-01-13 18:46:14.315 | WARNING  | scr.model:train:299 - SSN step rejected (consecutive=1/3)
2026-01-13 18:46:14.320 | DEBUG    | scr.ssn_tr:step:83 - 